In [ ]:
# 実データでの前処理

金融データをコピュラ分析用に変換

In [ ]:
import pandas as pd
import yfinance as yf
from scipy.stats import rankdata

# データ取得
symbols = ["^N225", "^GSPC"]  # 日経225, S&P500
print("データをダウンロード中...")
data = yf.download(symbols, start="2015-01-01", end="2024-01-01")['Close']
data.columns = ['Nikkei', 'SP500']

# 対数収益率
returns = np.log(data / data.shift(1)).dropna()

print(f"データ期間: {returns.index[0].strftime('%Y-%m-%d')} ～ {returns.index[-1].strftime('%Y-%m-%d')}")
print(f"観測数: {len(returns)}日")

# 基本統計
print("\n基本統計:")
print(returns.describe())

# コピュラ用の変換: 経験分布による一様分布化
def to_uniform_marginals(series):
    """データを経験分布により[0,1]区間に変換"""
    ranks = rankdata(series)
    return ranks / (len(ranks) + 1)

# 一様分布への変換
u_nikkei = to_uniform_marginals(returns['Nikkei'])
u_sp500 = to_uniform_marginals(returns['SP500'])

print(f"\n変換前データ範囲: {returns.min().min():.4f} ～ {returns.max().max():.4f}")
print(f"変換後データ範囲: {min(u_nikkei.min(), u_sp500.min()):.4f} ～ {max(u_nikkei.max(), u_sp500.max()):.4f}")

# 変換の可視化
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

ax1.scatter(returns['Nikkei'], returns['SP500'], alpha=0.6)
ax1.set_title('Original Returns')
ax1.set_xlabel('Nikkei Returns')
ax1.set_ylabel('S&P500 Returns')

ax2.scatter(u_nikkei, u_sp500, alpha=0.6, color='red')
ax2.set_title('Uniform Marginals (Copula Domain)')
ax2.set_xlabel('Nikkei Uniform')
ax2.set_ylabel('S&P500 Uniform')

plt.tight_layout()
plt.show()